# Parametrization of a simulation algorithm

In this basic example we are going to parameterize a simulation algorithm:

 - parameters linked to the number of points generated
 - the precision of the probability estimator
 - the sample storage strategy
 - using callbacks to monitor progress and stopping criteria.
 

In [148]:
from __future__ import print_function
import openturns as ot

In [149]:
# create the joint distribution of the parameters
distribution_R = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_F = ot.Normal(75e3, 5e3)
marginals = [distribution_R, distribution_F]
distribution = ot.ComposedDistribution(marginals)

In [150]:
# create the model
model = ot.SymbolicFunction(['R', 'F'], ['R-F/(_pi*100.0)'])

In [151]:
# create the event we want to estimate the probability
vect = ot.RandomVector(distribution)
G = ot.RandomVector(model, vect)
event = ot.Event(G, ot.Less(), 0.0)

In [152]:
# create a Monte Carlo algorithm
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)

In [153]:
# Criteria 1: Define the Maximum Coefficient of variation of the probability estimator
algo.setMaximumCoefficientOfVariation(0.05)

In [154]:
# Criteria 2: Define the Maximum Outer Sampling of the simulation
algo.setMaximumOuterSampling(int(2e4))

In [155]:
# HistoryStrategy to store the values of the probability used to draw the convergence graph

# Null strategy
algo.setConvergenceStrategy(ot.Null())

# Full strategy
algo.setConvergenceStrategy(ot.Full())

# Compact strategy: N points
N = 1000
algo.setConvergenceStrategy(ot.Compact(N))

In [156]:
# Use a callback to display the progress every 10%
def progress(p):
    if p >= progress.t:
        progress.t += 10.0
        print('progress=', p, '%')
    return False
progress.t = 10.0
algo.setProgressCallback(progress)

In [161]:
# Use a callback to stop the simulation
def stop():
    # here we never stop, but we could
    return False
algo.setStopCallback(stop)

In [158]:
algo.run()

In [159]:
# retrieve results
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)

Pf= 0.0
